# Importing 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn import set_config; set_config(display='diagram')
from package.etl import *
from package.train_baseline import TrainBaseline
from package.train_city import TrainCity
from package.train_store import TrainStore
from package.train_family import TrainFamily
import pandas as pd
import numpy as np
import requests
import joblib
import json

# Creating the Preprocessing